In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import dagshub
dagshub.init(repo_owner="gchit21",repo_name = "ML",mlflow=True)

In [ ]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

NameError: name 'pd' is not defined

# Dataset Info Gathering

In [ ]:
df = pd.read_table("/kaggle/input/house-prices-advanced-regression-techniques/train.csv",sep=",")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
test_dfcp = test_df.copy()

In [ ]:
from sklearn.model_selection import train_test_split

y = df["SalePrice"]
df.drop('SalePrice',axis=1,inplace=True)

X_train,X_test, y_train,y_test = train_test_split(df,y,test_size=0.2,random_state=42)

# Correlation function code

In [ ]:
def corr_matrix(X, y, threshold=0.8):
    corr_matrix = X.corr()
    
    high_corr_pairs = []

    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

    # To remove one feature from each highly correlated pair
    features_to_drop = []
    for feat1, feat2, _ in high_corr_pairs:
        # Compare correlation with target and add the feature with lower correlation to target to the drop list
        if abs(X[feat1].corr(y)) < abs(X[feat2].corr(y)):
            features_to_drop.append(feat1)
        else:
            features_to_drop.append(feat2)

    # Remove duplicates
    features_to_drop = list(set(features_to_drop))

    return features_to_drop,high_corr_pairs 


# data cleaning

In [ ]:
for col in X_train.columns:
    if df[col].dtype == "object":
        mode_val = df[col].mode()[0]
        X_train.fillna({col:mode_val}, inplace=True)
        X_test.fillna({col:mode_val},inplace=True)
        test_dfcp.fillna({col:mode_val},inplace=True)
    elif df[col].dtype in ["int64", "float64"]:
        mean_val = df[col].mean()
        X_train.fillna({col:mean_val}, inplace=True)
        X_test.fillna({col:mean_val},inplace=True)
        test_dfcp.fillna({col:mean_val},inplace=True)

In [ ]:
#dropping unnecessary columns from train set 
X_train.drop(['Alley','PoolQC'],axis=1,inplace=True)
X_train.drop(["Fence","MiscFeature"],axis=1,inplace=True)
X_train.drop(["MasVnrType","MasVnrArea"],axis=1,inplace=True)
X_train.drop(["FireplaceQu","Fireplaces"],axis=1,inplace=True)
X_train.drop("Id",axis=1,inplace=True)
X_train.drop("MiscVal",axis=1,inplace=True)
X_train.drop("LowQualFinSF",axis=1,inplace=True)
X_train.drop("PoolArea",axis=1,inplace=True)
X_train.drop("3SsnPorch",axis=1,inplace=True)


#dropping unnecessary columns from test 
test_dfcp.drop(["Alley","PoolQC","Fence","MiscFeature"],axis=1,inplace=True)
test_dfcp.drop(["MasVnrType","MasVnrArea","FireplaceQu","Fireplaces"],axis=1,inplace=True)
test_dfcp.drop(["Id","MiscVal","LowQualFinSF","PoolArea","3SsnPorch"],axis=1,inplace=True)

**get categorial columns**

In [ ]:
Cat_columns_less3 = [col for col in X_train.columns if df[col].dtype == 'object' and X_train[col].nunique()<=3]
Cat_columns_more3 = [col for col in X_train.columns if df[col].dtype == 'object'and X_train[col].nunique()>3]

**One-Hot Encoding and Woe encoding**

In [ ]:
from category_encoders.woe import WOEEncoder

In [ ]:
#make bins out of y so that woe is possible
median = y_train.median()
y_train_cp = (y_train>=median).astype(int).copy()

In [ ]:
woe_encoder = WOEEncoder(cols=Cat_columns_more3)

X_train[Cat_columns_more3] = woe_encoder.fit_transform(X_train[Cat_columns_more3],y_train_cp)
test_dfcp[Cat_columns_more3] = woe_encoder.transform(test_dfcp[Cat_columns_more3])

train_encoded = pd.get_dummies(X_train,columns=Cat_columns_less3,drop_first=True,dtype=int)
test_encoded = pd.get_dummies(X_test,columns=Cat_columns_less3,drop_first=True,dtype=int)
test_df_encoded = pd.get_dummies(test_dfcp,columns=Cat_columns_less3,drop_first=True,dtype=int)

In [ ]:
#duumies მეთოდმა შექმნა ეს სვეტი, 0-ებით სავსეა და უბრალოდ გადავაგდე, არაფრისმომცემია...
train_encoded.drop("Utilities_NoSeWa",axis=1,inplace=True)

In [ ]:
features,high =corr_matrix(train_encoded,y_train,threshold=0.7)
train_encoded_cp = train_encoded.drop(features,axis=1)
test_df_encoded_cp = test_df_encoded.drop(features,axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
train_scaled = pd.DataFrame(scaler.fit_transform(train_encoded_cp),columns=train_encoded_cp.columns)
test_df_scaled = pd.DataFrame(scaler.transform(test_df_encoded_cp),columns=test_df_encoded_cp.columns)

In [ ]:
import mlflow
from sklearn.feature_selection import RFE

# Prediction of Test.csv(the code above is for test.csv)

In [ ]:
logged_model = 'runs:/b2299001c241408289c8823c8ecd9126/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
rfe = RFE(estimator=loaded_model, n_features_to_select=15, step=2)
test_rfe = rfe.transform(test_df_scaled)
# Predict on your data.
Pred= loaded_model.predict(test_rfe)
end_pred=pd.DataFrame(Pred,index=test_df.Id,columns=['SalePrice'])
end_pred.to_csv('submission.csv',sep=',')
end_pred.head()
